In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import load_dataset, Dataset
from sklearn.preprocessing import LabelEncoder
import numpy as np
import evaluate
import torch

# Dataset uploading

In [3]:
train_set = load_dataset("csv", data_files="/kaggle/input/python-codes-time-complexity/train_set.csv")
test_set = load_dataset("csv", data_files="/kaggle/input/python-codes-time-complexity/test_set.csv")

train_labels = train_set['train']['complexity']
test_labels = test_set['train']['complexity']

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

# Checkpoint pool

In [4]:
checkpoints = ["microsoft/codebert-base", "neulab/codebert-python", "microsoft/graphcodebert-base", 
              "Salesforce/codet5-base", "Salesforce/codet5p-220m-py",
              "Salesforce/codet5-base-multi-sum", "microsoft/unixcoder-base"]

# Tokenizing

## Label tokenizing

In [5]:
labelEncoder = LabelEncoder()
labelEncoder.fit(train_labels)

LabelEncoder()

## Feature tokenizing

In [6]:
def tokenize_data(samples, tokenizer):
    tokenized = tokenizer(samples['code'], truncation=True)
    tokenized['labels'] = labelEncoder.transform(samples['complexity']) if samples['complexity'] else None
    return tokenized


def set_tokenizer(checkpoint):
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    X_train = train_set.map(lambda x: tokenize_data(x, tokenizer), batched=True)
    X_eval = test_set.map(lambda x: tokenize_data(x, tokenizer), batched=True)

    # Collator for batch padding
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    return tokenizer, data_collator, X_train, X_eval

# Model

In [7]:
def set_model(checkpoint):
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=7)
    return model

# Hyperparameters

In [8]:
def set_training_args(checkpoint, batch_size=32):
    training_args = TrainingArguments(output_dir=f"training_results/{checkpoint}/", 
                                      eval_strategy="epoch", logging_strategy="epoch",
                                      fp16=True, report_to='none', num_train_epochs=3, 
                                      per_device_train_batch_size=batch_size,
                                      per_device_eval_batch_size=batch_size)
    return training_args

# Evaluating

In [9]:
def compute_metrics(eval_preds):
    metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits[0], axis=-1) if isinstance(logits, tuple) else np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Trainer 

In [18]:
def mass_train_models(checkpoins):
    num_of_checkpoints = len(checkpoints[3:])
    
    for idx, checkpoint in enumerate(checkpoints[3:]):
        
        tokenizer, data_collator, train_set, eval_set = set_tokenizer(checkpoint)
        model = set_model(checkpoint)
        training_args = set_training_args(checkpoint=checkpoint, batch_size=8)
    
        trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_set['train'],
        eval_dataset=eval_set['train'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,)
        
        print(f"CHECKPOINT[{idx+1}/{num_of_checkpoints}]: {checkpoint}")
        trainer.train()
        


mass_train_models(checkpoints)

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

...

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at Salesforce/codet5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-18-8dbf30b72b33>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


CHECKPOINT[1/4]: Salesforce/codet5-base


Epoch,Training Loss,Validation Loss


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-8dbf30b72b33>", line 24, in <cell line: 24>
    mass_train_models(checkpoints)
  File "<ipython-input-18-8dbf30b72b33>", line 20, in mass_train_models
    trainer.train()
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2164, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2522, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs, num_items_in_batch)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 3688, in training_step
    self.accelerator.backward(loss, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py", line 2244, in backward
    self.scaler.scale(loss).backward(**kwargs)
  File

TypeError: object of type 'NoneType' has no len()

# Inference

In [ ]:
device = torch.cuda.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def predict(inputs):
    # Tokenizing inputs
    test_sample = tokenizer(inputs, return_tensors='pt')
    inputs = Dataset.from_dict({key: value.to(model.device) for key, value in test_sample.items()})

    # Predicting & decoding inputs
    preds = trainer.predict(test_dataset=inputs)
    preds = labelEncoder.inverse_transform(y=np.ravel(np.argmax(preds.predictions, axis=-1)))
    
    return preds

In [ ]:
test_sample = """
class Solution:
    def isValid(self, s: str) -> bool:
        bracketMap = {"(": ")", "[": "]", "{": "}"}
        openSet = set(["(", "[", "{"])
        stack = []
        for char in s:
            if char in openSet:
                stack.append(char)
            elif stack and char == bracketMap[stack[-1]]:
                stack.pop()
            else:
                return False
        return stack == []
        """

predict(test_sample)